# 可执行的UML建模 - 自然语言即模型

## 核心理念

传统UML是静态图形，需要转换成代码才能执行。  
而Agent+知识就是活的UML：
- 类图 = Agent知识定义
- 序列图 = 自然语言函数调用
- 状态图 = 知识中的状态流转规则
- 活动图 = 知识中的业务流程

**这个Notebook演示：画UML = 写知识 = 可执行程序**

In [1]:
# 初始化环境
import sys
import os
sys.path.insert(0, '/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need')

from core.react_agent_minimal import ReactAgentMinimal
from core.tools.create_agent_tool import CreateAgentTool
import json
from datetime import datetime

# 使用环境变量中的API配置
MODEL = "x-ai/grok-code-fast-1" #"deepseek-chat"  # x-ai/grok-code-fast-1
BASE_URL = "https://openrouter.ai/api/v1" # "https://api.deepseek.com/v1" 
API_KEY = os.getenv("OPENROUTER_API_KEY") # os.getenv("DEEPSEEK_API_KEY")

print("✅ 环境初始化完成")
print(f"📦 使用模型: {MODEL}")
print("⚡ 性能优化: 所有Agent使用stateful=False（无状态模式）")
print("   - 每次执行都是独立的，无历史负担")
print("   - 速度更快，内存占用更少")
print("   - 特别适合独立任务处理")

  ✅ 已加载11个环境变量: /home/guci/aiProjects/mda/pim-compiler/.env
✅ 环境初始化完成
📦 使用模型: x-ai/grok-code-fast-1
⚡ 性能优化: 所有Agent使用stateful=False（无状态模式）
   - 每次执行都是独立的，无历史负担
   - 速度更快，内存占用更少
   - 特别适合独立任务处理


# creator测试

In [3]:
creator_agent = ReactAgentMinimal(
    name="agent_creator",
    description="帮助业务人员创建Agent的智能助手",
    work_dir="/tmp/agent_creator",
    model=MODEL,
    base_url=BASE_URL,
    api_key=API_KEY,
    knowledge_files=[
        "/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/knowledge/agent_creator_self_knowledge.md"
    ],
    stateful=True,  # 每次创建都是独立的
    max_rounds=200
)

# 添加CreateAgentTool，这样Creator可以创建并直接使用Agent
create_tool = CreateAgentTool(work_dir="/tmp/agent_creator", parent_agent=creator_agent)
creator_agent.append_tool(create_tool)

x=creator_agent.execute(task="@hello")
print(x)

  ✅ 加载知识文件: execution_context_guide.md
  ✅ 加载知识文件: system_prompt_minimal.md
  ✅ 加载知识文件: agent_creator_self_knowledge.md
🚀 极简Agent已初始化 [agent_creator]
  📍 API: OpenRouter
  🤖 模型: x-ai/grok-code-fast-1
  🧠 Compact记忆: 70k tokens触发压缩
  ⚡ Compact记忆替代文件系统
  📚 知识文件: 3个
  ✨ Compact即注意力机制
  ➕ 已添加工具: create_agent
  📖 加载语义记忆（后备）: /tmp/agent_creator/agent.md

[agent_creator] 执行任务...
[agent_creator] 📝 任务: @hello...


[agent_creator] ⏰ 时间: 2025-09-17 06:54:18.832988
[agent_creator] ============================================================

[agent_creator] 🤔 思考第1轮...
[agent_creator] 💭 思考: 你好123
今天天气不错...

[agent_creator] ✅ 任务完成（第1轮）
你好123
今天天气不错


In [4]:

business_description="""
创建一个知识图谱生成Agent：

# 需求
1. 功能：从文本生成Turtle格式RDF知识图谱
2. 输出：必须将RDF保存到.ttl文件
3. 对agent必须做符号主义验证

@创建并测试Agent(需求)
"""
x=creator_agent.execute(task=business_description)
print(x)


  📖 加载语义记忆（后备）: /tmp/agent_creator/agent.md

[agent_creator] 执行任务...
[agent_creator] 📝 任务: 
创建一个知识图谱生成Agent：

# 需求
1. 功能：从文本生成Turtle格式RDF知识图谱
2. 输出：必须将RDF保存到.ttl文件
3. 对agent必须做符号主义验证

@创建并测试A...
[agent_creator] ⏰ 时间: 2025-09-17 06:55:21.060688
[agent_creator] ============================================================

[agent_creator] 🤔 思考第1轮...

[agent_creator] 🔧 调用工具: context
   [agent_creator] 📝 action: init_project
   [agent_creator] 📝 goal: 创建一个知识图谱生成Agent：从文本生成Turtle格式RDF知识图谱，输出：必须将RDF保存到.ttl文件，对agent必须做符号主义验证
   [agent_creator] ✅ 结果: ✅ 项目已初始化: 创建一个知识图谱生成Agent：从文本生成Turtle格式RDF知识图谱，输出：必须将RDF保存到.ttl文件，对agent必须做符号主义验证

[agent_creator] 🤔 思考第2轮...

[agent_creator] 🔧 调用工具: context
   [agent_creator] 📝 action: add_tasks
   [agent_creator] 📝 tasks: ['分析需求并提取关键信息', '生成初始知识文件', '创建Agent实例', '执行测试和符号主义验证', '处理反馈循环结果']
   [agent_creator] ✅ 结果: ✅ 添加了 5 个任务

[agent_creator] 🤔 思考第3轮...

[agent_creator] 🔧 调用工具: context
   [agent_creator] 📝 action: start_task
   [agent_creator] 📝 task: 分析需求并提取关键

In [8]:

from typing import cast
rdf_agent:ReactAgentMinimal=cast(ReactAgentMinimal, creator_agent.function_instances[-1])

In [9]:
rdf_agent.description

'从文本生成Turtle格式RDF知识图谱的Agent'

## 1. UML类图 → Agent知识定义

传统方式：画UML类图 → 生成代码框架 → 实现业务逻辑  
Agent方式：定义知识字符串 → 直接可执行

### 客户服务类（CustomerService）

In [ ]:
# UML类定义 - 客户服务
customer_uml_knowledge = """
# 客户服务Agent - UML类定义

## 类名：CustomerService
## 职责：管理客户信息和会员等级

## 属性（状态）
- customers: 客户信息映射表
- membership_levels: 会员等级定义
- data_file: /tmp/uml_demo/customers.json

## 方法（行为）

### get_customer_info(customer_id)
获取客户信息，返回客户详情和会员等级。
如果文件不存在，初始化示例数据：
- CUST001: 张三，VIP会员，8折
- CUST002: 李四，普通会员，9折
- CUST003: 王五，非会员，无折扣

### update_membership(customer_id, new_level)
更新客户会员等级

### add_customer(id, name, level)
添加新客户

## 业务规则
- VIP会员享受8折优惠
- 普通会员享受9折优惠
- 非会员无折扣
- 客户ID格式：CUST + 3位数字
"""

# 创建客户服务Agent - 使用load_knowledge_str
customer_agent = ReactAgentMinimal(
    name="customer_service",
    description="客户信息和会员管理服务",
    work_dir="/tmp/uml_demo/customers",
    model=MODEL,
    base_url=BASE_URL,
    api_key=API_KEY,
    stateful=False  # 无状态模式，每次执行都是独立的
)
customer_agent.load_knowledge_str(customer_uml_knowledge)

print("✅ CustomerService类已创建（可执行，无状态模式）")

In [5]:
x=customer_agent.execute(task="列出所有的客户")
print(x)


[customer_service] 执行任务...
[customer_service] 📝 任务: 列出所有的客户...
[customer_service] ⏰ 时间: 2025-09-15 17:56:43.297106
[customer_service] ============================================================

[customer_service] 🤔 思考第1轮...

[customer_service] 🔧 调用工具: execute_command
   [customer_service] 📝 command: ls -la /tmp/uml_demo/customers.json
   [customer_service] ✅ 结果: 命令执行完成

[customer_service] 🤔 思考第2轮...

[customer_service] 🔧 调用工具: execute_command
   [customer_service] 📝 command: pwd && ls -la customers.json
   [customer_service] ✅ 结果: /tmp/uml_demo/customers
-rw-rw-r-- 1 guci guci 418  9月 15 17:55 customers.json


[customer_service] 🤔 思考第3轮...

[customer_service] 🔧 调用工具: read_file
   [customer_service] 📝 file_path: customers.json
   [customer_service] ✅ 结果: {
  "customers": {
    "CUST001": {
      "name": "张三",
      "level": "VIP会员",
      "discount": 0.8
    },
    "CUST002": {
      "name": "李四",
    

[customer_service] 🤔 思考第4轮...
[customer_service] 💭 思考: 以下是所有客户的列表：

- **CUST001**

### 产品服务类（ProductService）

In [ ]:
# UML类定义 - 产品服务
product_uml_knowledge = """
# 产品服务Agent - UML类定义

## 类名：ProductService
## 职责：管理产品信息和价格

## 属性（状态）
- products: 产品目录
- categories: 产品分类
- data_file: /tmp/uml_demo/products.json

## 方法（行为）

### get_product_info(product_id)
获取产品信息和价格。
如果文件不存在，初始化示例数据：
- PROD001: iPhone 15 Pro, 8999元
- PROD002: AirPods Pro, 1999元
- PROD003: MacBook Pro, 19999元

### update_price(product_id, new_price)
更新产品价格

### add_product(id, name, price, category)
添加新产品

## 业务规则
- 产品ID格式：PROD + 3位数字
- 价格必须大于0
- 产品必须属于有效分类
"""

# 创建产品服务Agent
product_agent = ReactAgentMinimal(
    name="product_service",
    description="产品信息和价格管理服务",
    work_dir="/tmp/uml_demo/products",
    model=MODEL,
    base_url=BASE_URL,
    api_key=API_KEY,
    stateful=False  # 无状态模式，每次执行都是独立的
)
product_agent.load_knowledge_str(product_uml_knowledge)

print("✅ ProductService类已创建（可执行，无状态模式）")

In [7]:
x=product_agent.execute(task="列出所有的产品")
print(x)


[product_service] 执行任务...
[product_service] 📝 任务: 列出所有的产品...
[product_service] ⏰ 时间: 2025-09-15 17:57:51.897234
[product_service] ============================================================

[product_service] 🤔 思考第1轮...

[product_service] 🔧 调用工具: read_file
   [product_service] 📝 file_path: products.json
   [product_service] ✅ 结果: 文件不存在: products.json

[product_service] 🤔 思考第2轮...

[product_service] 🔧 调用工具: write_file
   [product_service] 📝 file_path: products.json
   [product_service] 📝 content: {
  "products": {
    "PROD001": {
      "name": "iPhone 15 Pro",
      "price": 8999
    },
    "PR...
   [product_service] ✅ 结果: 文件已写入: products.json

[product_service] 🤔 思考第3轮...

[product_service] 🔧 调用工具: read_file
   [product_service] 📝 file_path: products.json
   [product_service] ✅ 结果: {
  "products": {
    "PROD001": {
      "name": "iPhone 15 Pro",
      "price": 8999
    },
    "PROD002": {
      "name": "AirPods Pro",
      "pric

[product_service] 🤔 思考第4轮...
[product_service] 💭 思考

### 库存服务类（InventoryService）

In [ ]:
# UML类定义 - 库存服务
inventory_uml_knowledge = """
# 库存服务Agent - UML类定义

## 类名：InventoryService
## 职责：管理产品库存

## 属性（状态）
- inventory: 库存数量映射
- reserved: 预留库存
- data_file: /tmp/uml_demo/inventory.json

## 方法（行为）

### check_stock(product_id, quantity)
检查库存是否充足。
如果文件不存在，初始化示例数据：
- PROD001: 库存50台
- PROD002: 库存100个
- PROD003: 库存20台

### reserve_stock(product_id, quantity)
预留库存（创建订单时）

### deduct_stock(product_id, quantity)
扣减库存（订单确认后）

### release_stock(product_id, quantity)
释放库存（订单取消时）

## 业务规则
- 库存不能为负数
- 预留库存在订单确认前不能被其他订单使用
- 订单取消时必须释放预留库存
"""

# 创建库存服务Agent
inventory_agent = ReactAgentMinimal(
    name="inventory_service",
    description="库存管理服务",
    work_dir="/tmp/uml_demo/inventory",
    model=MODEL,
    base_url=BASE_URL,
    api_key=API_KEY,
    stateful=False  # 无状态模式，每次执行都是独立的
)
inventory_agent.load_knowledge_str(inventory_uml_knowledge)

print("✅ InventoryService类已创建（可执行，无状态模式）")

In [9]:
x=inventory_agent.execute(task="列出所有库存")
print(x)


[inventory_service] 执行任务...
[inventory_service] 📝 任务: 列出所有库存...
[inventory_service] ⏰ 时间: 2025-09-15 17:58:47.835427
[inventory_service] ============================================================

[inventory_service] 🤔 思考第1轮...

[inventory_service] 🔧 调用工具: read_file
   [inventory_service] 📝 file_path: inventory.json
   [inventory_service] ✅ 结果: 文件不存在: inventory.json

[inventory_service] 🤔 思考第2轮...

[inventory_service] 🔧 调用工具: write_file
   [inventory_service] 📝 file_path: inventory.json
   [inventory_service] 📝 content: {"PROD001": 50, "PROD002": 100, "PROD003": 20}
   [inventory_service] ✅ 结果: 文件已写入: inventory.json

[inventory_service] 🤔 思考第3轮...

[inventory_service] 🔧 调用工具: read_file
   [inventory_service] 📝 file_path: inventory.json
   [inventory_service] ✅ 结果: {"PROD001": 50, "PROD002": 100, "PROD003": 20}

[inventory_service] 🤔 思考第4轮...
[inventory_service] 💭 思考: 所有库存如下：

- PROD001: 50 台
- PROD002: 100 个  
- PROD003: 20 台

（注：由于库存文件不存在，已自动初始化为示例数据）...

[inventory_service] ✅ 任务完成

## 2. UML序列图 → 业务流程编排

传统方式：画序列图 → 编写调用代码  
Agent方式：定义流程知识 → 自然语言执行

### 订单服务类（OrderService） - 主Agent

In [ ]:
# UML序列图定义 - 订单处理流程
order_uml_knowledge = """
# 订单服务Agent - 自然语言流程定义

## 类名：OrderService
## 职责：我是订单管家，负责协调大家完成订单

## 我认识的朋友们（可以对话的服务）
- customer_service: 客户服务，知道谁是VIP
- product_service: 产品服务，知道价格
- inventory_service: 库存服务，管理库存

## 创建订单时我会这样做

当有人说"我要买东西"时，我会开始一段对话：

1. 我先问customer_service："这位客户是谁？有什么优惠吗？"
   他会告诉我客户的身份和折扣。

2. 然后问product_service："这个产品多少钱？"
   他会告诉我准确的价格。

3. 接着问inventory_service："还有货吗？够不够？"
   他会确认库存情况。

4. 我自己算算总价：
   - 原价乘以数量
   - 应用会员折扣
   - 看看有没有满减活动
   - 满1000减100，满5000减500

5. 生成一个订单号，格式是：ORD-今天日期-序号

6. 告诉inventory_service："东西卖出去了，减少库存吧。"

7. 把订单保存起来，放在orders.json文件里。

8. 最后告诉客户："订单创建成功啦！"

## 如果出了问题

- 客户不存在："先去注册吧"
- 产品没货了："换个产品或者等等吧"
- 价格算错了："我再算一遍"

## 订单的生命周期

订单就像包裹一样有自己的旅程：
- 刚创建：待支付（pending）
- 付了钱：已支付（paid）
- 发货了：配送中（shipped）
- 收到了：已完成（completed）
- 不要了：已取消（cancelled）
"""

# 创建订单服务Agent（主Agent）
order_agent = ReactAgentMinimal(
    name="order_service",
    description="订单处理主服务，协调其他服务完成订单流程",
    work_dir="/tmp/uml_demo/orders",
    model=MODEL,
    base_url=BASE_URL,
    api_key=API_KEY,
    stateful=False  # 无状态模式，每个订单处理都是独立的
)
order_agent.load_knowledge_str(order_uml_knowledge)

# 组合服务 - 将其他Agent注册为自然语言函数
order_agent.add_function(customer_agent)
order_agent.add_function(product_agent)
order_agent.add_function(inventory_agent)

print("✅ OrderService主Agent已创建（无状态模式）")
print("📦 已注册子服务：customer_service, product_service, inventory_service")
print("⚡ 性能优化：每个订单处理都是独立的，无历史负担")

## 3. 执行UML模型 - 自然语言即接口

现在我们的UML模型是活的，可以直接执行！

### 初始化测试数据

在执行订单之前，我们可以预先初始化一些测试数据，让系统更快响应：

In [ ]:
# 初始化订单系统测试数据
import json
import os

def init_test_data():
    """初始化测试数据，加速后续执行"""
    
    # 创建目录
    dirs = [
        "/tmp/uml_demo/customers",
        "/tmp/uml_demo/products", 
        "/tmp/uml_demo/inventory",
        "/tmp/uml_demo/orders"
    ]
    
    for dir_path in dirs:
        os.makedirs(dir_path, exist_ok=True)
    
    # 初始化客户数据
    customers_data = {
        "customers": {
            "CUST001": {
                "name": "张三",
                "level": "VIP会员",
                "discount": 0.8,
                "total_orders": 10,
                "registration_date": "2023-01-01"
            },
            "CUST002": {
                "name": "李四",
                "level": "普通会员",
                "discount": 0.9,
                "total_orders": 5,
                "registration_date": "2023-06-15"
            },
            "CUST003": {
                "name": "王五",
                "level": "非会员",
                "discount": 1.0,
                "total_orders": 1,
                "registration_date": "2024-01-01"
            },
            "CORP001": {
                "name": "科技有限公司",
                "level": "企业客户",
                "discount": 0.85,
                "total_orders": 50,
                "registration_date": "2022-01-01"
            }
        }
    }
    
    with open("/tmp/uml_demo/customers/customers.json", "w", encoding="utf-8") as f:
        json.dump(customers_data, f, ensure_ascii=False, indent=2)
    
    # 初始化产品数据
    products_data = {
        "products": {
            "PROD001": {
                "name": "iPhone 15 Pro",
                "price": 8999,
                "category": "手机",
                "description": "最新款iPhone Pro"
            },
            "PROD002": {
                "name": "AirPods Pro",
                "price": 1999,
                "category": "耳机",
                "description": "降噪耳机"
            },
            "PROD003": {
                "name": "MacBook Pro",
                "price": 19999,
                "category": "笔记本",
                "description": "专业笔记本电脑"
            },
            "PROD004": {
                "name": "iPad Pro",
                "price": 6999,
                "category": "平板",
                "description": "专业平板电脑"
            }
        }
    }
    
    with open("/tmp/uml_demo/products/products.json", "w", encoding="utf-8") as f:
        json.dump(products_data, f, ensure_ascii=False, indent=2)
    
    # 初始化库存数据
    inventory_data = {
        "PROD001": 50,
        "PROD002": 100,
        "PROD003": 20,
        "PROD004": 30
    }
    
    with open("/tmp/uml_demo/inventory/inventory.json", "w", encoding="utf-8") as f:
        json.dump(inventory_data, f, ensure_ascii=False, indent=2)
    
    # 初始化空订单文件
    orders_data = {"orders": [], "next_id": 1}
    
    with open("/tmp/uml_demo/orders/orders.json", "w", encoding="utf-8") as f:
        json.dump(orders_data, f, ensure_ascii=False, indent=2)
    
    print("✅ 测试数据初始化完成！")
    print("\n📊 已创建的测试数据：")
    print("├── 客户数据")
    print("│   ├── CUST001: 张三 (VIP会员, 8折)")
    print("│   ├── CUST002: 李四 (普通会员, 9折)")
    print("│   ├── CUST003: 王五 (非会员, 无折扣)")
    print("│   └── CORP001: 科技有限公司 (企业客户, 8.5折)")
    print("├── 产品数据")
    print("│   ├── PROD001: iPhone 15 Pro (¥8999)")
    print("│   ├── PROD002: AirPods Pro (¥1999)")
    print("│   ├── PROD003: MacBook Pro (¥19999)")
    print("│   └── PROD004: iPad Pro (¥6999)")
    print("└── 库存数据")
    print("    ├── iPhone: 50台")
    print("    ├── AirPods: 100个")
    print("    ├── MacBook: 20台")
    print("    └── iPad: 30台")
    
    return True

# 执行初始化
if init_test_data():
    print("\n💡 提示：现在Agent可以直接读取这些数据，无需每次创建！")
    print("⚡ 这会让订单处理更快，因为避免了文件创建的开销。")

In [11]:
# 清理测试环境
import shutil
if os.path.exists("/tmp/uml_demo"):
    shutil.rmtree("/tmp/uml_demo")
print("✅ 测试环境已清理")

✅ 测试环境已清理


### 场景1：简单的订单创建

In [12]:
# 执行UML序列图 - 创建订单
print("🚀 执行订单创建流程...\n")
print("📝 业务请求：为VIP客户张三创建订单")
print("-" * 50)

result = order_agent.execute(
    task="为客户CUST001创建订单，购买iPhone 15 Pro一台和AirPods Pro两个"
)

print("\n✅ 执行结果：")
print(result[:500] if len(result) > 500 else result)

🚀 执行订单创建流程...

📝 业务请求：为VIP客户张三创建订单
--------------------------------------------------

[order_service] 执行任务...
[order_service] 📝 任务: 为客户CUST001创建订单，购买iPhone 15 Pro一台和AirPods Pro两个...
[order_service] ⏰ 时间: 2025-09-15 18:00:08.281079
[order_service] ============================================================

[order_service] 🤔 思考第1轮...

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: init_project
   [order_service] 📝 goal: 为客户CUST001创建订单，购买iPhone 15 Pro一台和AirPods Pro两个
   [order_service] ✅ 结果: ✅ 项目已初始化: 为客户CUST001创建订单，购买iPhone 15 Pro一台和AirPods Pro两个

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: add_tasks
   [order_service] 📝 tasks: ['验证客户信息', '获取产品价格', '检查库存', '计算总价', '生成订单号', '扣减库存', '保存订单']
   [order_service] ✅ 结果: ✅ 添加了 7 个任务

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: start_task
   [order_service] 📝 task: 验证客户信息
   [order_service] ✅ 结果: ✅ 任务 验证客户信息 标记为执行中

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: set_state
   [order_

### 场景2：更自然的业务请求

In [13]:
# 使用更自然的语言
print("🚀 使用自然语言请求...\n")
print("📝 业务请求：张三想买个MacBook")
print("-" * 50)

result = order_agent.execute(
    task="客户CUST001（张三）想买一台MacBook Pro"
)

print("\n✅ 执行结果：")
print(result[:500] if len(result) > 500 else result)

🚀 使用自然语言请求...

📝 业务请求：张三想买个MacBook
--------------------------------------------------

[order_service] 执行任务...
[order_service] 📝 任务: 客户CUST001（张三）想买一台MacBook Pro...
[order_service] ⏰ 时间: 2025-09-15 18:03:54.727189
[order_service] ============================================================

[order_service] 🤔 思考第1轮...

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: init_project
   [order_service] 📝 goal: 为客户CUST001创建订单，购买MacBook Pro一台
   [order_service] ✅ 结果: ✅ 项目已初始化: 为客户CUST001创建订单，购买MacBook Pro一台

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: add_tasks
   [order_service] 📝 tasks: ['验证客户信息', '获取产品价格', '检查库存', '计算总价', '生成订单号', '扣减库存', '保存订单']
   [order_service] ✅ 结果: ✅ 添加了 7 个任务

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: start_task
   [order_service] 📝 task: 验证客户信息
   [order_service] ✅ 结果: ✅ 任务 验证客户信息 标记为执行中

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: set_state
   [order_service] 📝 state: 正在验证客户CUST001的信息
   [order_servi

### 场景3：查询订单状态

In [14]:
# 查询订单
print("🔍 查询订单信息...\n")

result = order_agent.execute(
    task="查看所有订单的状态和总金额"
)

print("📊 查询结果：")
print(result[:800] if len(result) > 800 else result)

🔍 查询订单信息...


[order_service] 执行任务...
[order_service] 📝 任务: 查看所有订单的状态和总金额...
[order_service] ⏰ 时间: 2025-09-15 18:05:59.480493
[order_service] ============================================================

[order_service] 🤔 思考第1轮...

[order_service] 🔧 调用工具: read_file
   [order_service] 📝 file_path: /tmp/uml_demo/orders/orders.json
   [order_service] ✅ 结果: [{"order_id":"ORD-20250915-001","customer_id":"CUST001","customer_name":"张三","items":[{"product":"iPhone 15 Pro","price":8999,"quantity":1},{"product"

[order_service] 🤔 思考第2轮...
[order_service] 💭 思考: ### 所有订单状态和总金额

**订单列表：**

1. **订单号**：ORD-20250915-001  
   - **状态**：待支付 (pending)  
   - **总金额**：9897.6元  
   - **商品**：iPhone 15 Pro × 1, AirPods Pro × 2

2. **订单号**：ORD-20250915-002  
   - **状态**：待支...

[order_service] ✅ 任务完成（第2轮）
📊 查询结果：
### 所有订单状态和总金额

**订单列表：**

1. **订单号**：ORD-20250915-001  
   - **状态**：待支付 (pending)  
   - **总金额**：9897.6元  
   - **商品**：iPhone 15 Pro × 1, AirPods Pro × 2

2. **订单号**：ORD-20250915-002  
   - **状态**：待支付 

## 4. 动态修改UML模型

传统UML需要重新画图和生成代码。  
Agent UML只需要修改知识字符串！

In [15]:
# 动态修改业务规则 - 添加企业客户支持
enhanced_order_knowledge = order_uml_knowledge + """

## 企业客户特殊规则（新增）
- 企业客户ID以CORP开头
- 企业客户享受批量采购折扣：
  - 10件以上：额外95折
  - 50件以上：额外9折
  - 100件以上：额外85折
- 企业订单自动开具增值税发票
"""

# 重新加载知识
order_agent.load_knowledge_str(enhanced_order_knowledge)
print("✅ UML模型已更新：添加企业客户支持")

  ✅ 动态加载知识: dynamic_knowledge
✅ UML模型已更新：添加企业客户支持


In [16]:
# 测试新功能
print("🏢 测试企业采购...\n")

result = order_agent.execute(
    task="企业客户CORP001采购15台iPhone作为年会奖品"
)

print("📊 企业订单结果：")
print(result[:500] if len(result) > 500 else result)

🏢 测试企业采购...


[order_service] 执行任务...
[order_service] 📝 任务: 企业客户CORP001采购15台iPhone作为年会奖品...
[order_service] ⏰ 时间: 2025-09-15 18:06:29.199891
[order_service] ============================================================

[order_service] 🤔 思考第1轮...

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: init_project
   [order_service] 📝 goal: 为企业客户CORP001创建订单，采购15台iPhone作为年会奖品
   [order_service] ✅ 结果: ✅ 项目已初始化: 为企业客户CORP001创建订单，采购15台iPhone作为年会奖品

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: add_tasks
   [order_service] 📝 tasks: ['验证客户信息', '获取产品价格', '检查库存', '计算总价', '生成订单号', '扣减库存', '保存订单']
   [order_service] ✅ 结果: ✅ 添加了 7 个任务

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: start_task
   [order_service] 📝 task: 验证客户信息
   [order_service] ✅ 结果: ✅ 任务 验证客户信息 标记为执行中

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: set_state
   [order_service] 📝 state: 正在验证企业客户CORP001的信息
   [order_service] ✅ 结果: ✅ 已设置 当前状态 = 正在验证企业客户CORP001的信息

[order_service] 🔧 调用

## 5. UML组合模式 - Agent编排

展示如何通过组合Agent创建复杂系统

In [ ]:
# 创建一个供应链协调Agent - 更高层的编排
supply_chain_knowledge = """
# 供应链协调Agent

## 职责：协调订单、库存和物流

## 可用服务
- order_service: 订单处理服务
- logistics_service: 物流服务（模拟）
- supplier_service: 供应商服务（模拟）

## 智能补货流程
1. 监控库存水平
2. 预测需求趋势
3. 自动生成采购订单
4. 协调供应商发货
5. 更新库存系统

## 订单履行流程
1. 接收订单请求
2. 调用order_service创建订单
3. 安排物流配送
4. 跟踪订单状态
5. 处理异常情况
"""

# 创建供应链Agent
supply_chain_agent = ReactAgentMinimal(
    name="supply_chain",
    description="供应链协调服务",
    work_dir="/tmp/uml_demo/supply_chain",
    model=MODEL,
    base_url=BASE_URL,
    api_key=API_KEY,
    stateful=False  # 无状态模式，每次供应链任务都是独立的
)
supply_chain_agent.load_knowledge_str(supply_chain_knowledge)

# 将订单Agent作为子服务
supply_chain_agent.add_function(order_agent)

print("✅ 供应链协调Agent已创建（无状态模式）")
print("🔗 已集成order_service（包含其所有子服务）")

In [18]:
# 测试多层Agent协作
print("🌐 测试供应链协调...\n")

result = supply_chain_agent.execute(
    task="处理客户CUST002的紧急订单：5台iPhone，要求明天送达"
)

print("📦 供应链处理结果：")
print(result[:600] if len(result) > 600 else result)

🌐 测试供应链协调...


[supply_chain] 执行任务...
[supply_chain] 📝 任务: 处理客户CUST002的紧急订单：5台iPhone，要求明天送达...
[supply_chain] ⏰ 时间: 2025-09-15 18:09:01.605442
[supply_chain] ============================================================

[supply_chain] 🤔 思考第1轮...

[supply_chain] 🔧 调用工具: context
   [supply_chain] 📝 action: init_project
   [supply_chain] 📝 goal: 处理客户CUST002的紧急订单：5台iPhone，要求明天送达
   [supply_chain] ✅ 结果: ✅ 项目已初始化: 处理客户CUST002的紧急订单：5台iPhone，要求明天送达

[supply_chain] 🔧 调用工具: context
   [supply_chain] 📝 action: add_tasks
   [supply_chain] 📝 tasks: ['检查库存可用性', '创建订单', '安排物流配送', '跟踪订单状态']
   [supply_chain] ✅ 结果: ✅ 添加了 4 个任务

[supply_chain] 🔧 调用工具: context
   [supply_chain] 📝 action: set_data
   [supply_chain] 📝 key: 成功判定条件
   [supply_chain] 📝 value: 5.0
   [supply_chain] ✅ 结果: ✅ 已设置 成功判定条件 = 5.0

[supply_chain] 🔧 调用工具: context
   [supply_chain] 📝 action: start_task
   [supply_chain] 📝 task: 检查库存可用性
   [supply_chain] ✅ 结果: ✅ 任务 检查库存可用性 标记为执行中

[supply_chain] 🔧 调用工具: context
   [supply_chain] 📝 action

## 6. 总结：UML的未来

### 传统UML vs 可执行UML

| 方面 | 传统UML | Agent可执行UML |
|------|---------|----------------|
| 形式 | 静态图形 | 动态知识 |
| 执行 | 需要代码生成 | 直接执行 |
| 修改 | 重新画图+编码 | 修改知识字符串 |
| 接口 | 严格API | 自然语言 |
| 组合 | 编译时绑定 | 运行时组合 |
| 智能 | 无 | 自主理解和执行 |

### 核心洞察

1. **知识即模型**：UML图可以用知识字符串表达
2. **模型即程序**：加载知识的Agent就是可执行程序
3. **自然语言即接口**：无需定义API，直接用自然语言
4. **组合即系统**：通过add_function组合出复杂系统

### 这改变了什么？

- **建模即编程**：画UML就是在编程
- **业务人员可编程**：用自然语言描述即可
- **极致的敏捷**：修改知识立即生效
- **真正的MDA**：模型驱动架构不再是梦想

In [ ]:
# 展示系统架构
print("🏗️ 当前系统架构")
print("=" * 60)
print("""
供应链协调Agent
    ├── 订单服务Agent（主Agent）
    │   ├── 客户服务Agent
    │   ├── 产品服务Agent
    │   └── 库存服务Agent
    ├── 物流服务Agent（可扩展）
    └── 供应商服务Agent（可扩展）

特点：
- 每层都是独立的Agent
- 通过自然语言函数通信
- 知识定义行为
- 运行时动态组合
""")
print("=" * 60)
print("\n✨ 这就是可执行的UML建模！")

## 7. 交互式建模体验

你可以修改下面的知识定义，立即看到效果！

In [ ]:
# 交互式UML建模 - 修改这里的知识定义
interactive_knowledge = """
# 促销活动Agent

## 职责：管理促销活动和优惠券

## 当前促销活动
- 双11特惠：全场8折
- 新用户礼包：首单减200
- 会员日：VIP额外9折

## 优惠规则
1. 优惠可以叠加
2. 最高优惠不超过50%
3. 优先使用大额优惠券

## 计算最优优惠方案
根据客户类型和订单金额，自动计算最优惠的方案。
"""

# 创建促销Agent
promo_agent = ReactAgentMinimal(
    name="promotion_service",
    description="促销活动管理服务",
    work_dir="/tmp/uml_demo/promotions",
    model=MODEL,
    base_url=BASE_URL,
    api_key=API_KEY,
    stateful=False  # 无状态模式，每次促销计算都是独立的
)
promo_agent.load_knowledge_str(interactive_knowledge)

# 将促销服务添加到订单服务
order_agent.add_function(promo_agent)

print("✅ 促销服务已添加到订单系统（无状态模式）")
print("🎉 现在订单会自动应用促销优惠！")
print("⚡ 性能优化：每次促销计算都是干净上下文")

In [20]:
# 测试带促销的订单
print("🎁 测试促销优惠...\n")

result = order_agent.execute(
    task="双11活动，VIP客户CUST001购买MacBook Pro"
)

print("💰 优惠订单结果：")
print(result[:600] if len(result) > 600 else result)

🎁 测试促销优惠...


[order_service] 执行任务...
[order_service] 📝 任务: 双11活动，VIP客户CUST001购买MacBook Pro...
[order_service] ⏰ 时间: 2025-09-15 18:11:18.057017
[order_service] ============================================================

[order_service] 🤔 思考第1轮...

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: init_project
   [order_service] 📝 goal: 为VIP客户CUST001创建双11活动订单，购买MacBook Pro
   [order_service] ✅ 结果: ✅ 项目已初始化: 为VIP客户CUST001创建双11活动订单，购买MacBook Pro

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: add_tasks
   [order_service] 📝 tasks: ['验证客户信息', '获取促销信息', '获取产品价格', '检查库存', '计算总价', '生成订单号', '扣减库存', '保存订单']
   [order_service] ✅ 结果: ✅ 添加了 8 个任务

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: start_task
   [order_service] 📝 task: 验证客户信息
   [order_service] ✅ 结果: ✅ 任务 验证客户信息 标记为执行中

[order_service] 🔧 调用工具: context
   [order_service] 📝 action: set_state
   [order_service] 📝 state: 正在验证VIP客户CUST001的信息
   [order_service] ✅ 结果: ✅ 已设置 当前状态 = 正在验证VIP客户CUST001的信息

[

## 结语

**这个Notebook展示了：**

1. ✅ UML类图 = Agent知识定义
2. ✅ UML序列图 = 自然语言函数调用流程
3. ✅ UML状态图 = 知识中的状态规则
4. ✅ UML活动图 = 知识中的业务流程
5. ✅ 模型可以直接执行，无需代码生成
6. ✅ 通过load_knowledge_str动态加载和修改
7. ✅ Agent通过add_function自由组合

**核心价值：**

- 🎯 **建模即编程**：UML不再是文档，而是程序
- 🚀 **零代码开发**：业务人员可以直接建模
- 🔄 **极致敏捷**：修改立即生效，无需编译
- 🧩 **灵活组合**：Agent像乐高一样组装

**这就是软件开发的未来：自然语言驱动的可执行建模！**